In [2]:
import os
from PIL import Image
from tqdm import tqdm

# fhd 폴더 정리 함수
def clean_fhd_folder(fhd_folder, source_files):
    """fhd 폴더 내 .webp 파일 중 원본 파일 목록과
    매칭되지 않는 파일을 삭제한다."""
    if os.path.exists(fhd_folder):
        for file in os.listdir(fhd_folder):
            if file.lower().endswith(".webp"):
                source_name = os.path.splitext(file)[0]
                if source_name not in source_files:
                    os.remove(os.path.join(fhd_folder, file))
                    print(f"Deleted: {os.path.join(fhd_folder, file)} (no matching source file)")

# 리사이즈 및 WebP 저장 함수
def resize_image(file_path, output_path, max_size=1920):
    """원본 비율을 유지하면서 긴 변을 max_size(px)로 맞춘다.
    이미 원본이 더 작으면 리사이즈 없이 그대로 WebP로 저장."""
    with Image.open(file_path) as img:
        width, height = img.size

        # 이미 가로, 세로 모두 max_size 이하라면 리사이즈 없이 저장
        if width <= max_size and height <= max_size:
            img.save(output_path, format="WEBP", quality=85)
            return

        # 긴 변이 max_size가 되도록 비율 맞춰 리사이즈
        if width > height:
            new_width = max_size
            new_height = int(height * (max_size / float(width)))
        else:
            new_height = max_size
            new_width = int(width * (max_size / float(height)))

        resized_img = img.resize((new_width, new_height), Image.Resampling.LANCZOS)
        resized_img.save(output_path, format="WEBP", quality=85)

# 최상위 폴더(재귀적으로 하위 폴더 순회)
base_dir = "./src/assets"

# os.walk를 사용해 모든 하위 디렉토리를 깊이와 상관없이 순회
# root: 현재 디렉토리 경로, dirs: root 안의 하위 디렉토리 목록, files: root 안의 파일 목록
for root, dirs, files in os.walk(base_dir):
    # 'fhd' 디렉토리 자체(및 그 안)를 다시 처리하는 것은 피하려면 건너뛴다.
    # (필요에 따라 제거하거나 수정 가능)
    if os.path.basename(root) == "fhd":
        continue

    # 현재 디렉토리에 있는 원본 파일 이름 목록 생성 (확장자는 소문자로 비교)
    source_files = {
        os.path.splitext(f)[0]
        for f in files
        if os.path.splitext(f)[1].lower() in (".jpeg", ".jpg", ".png")
    }

    # fhd 폴더 경로 정의
    fhd_folder = os.path.join(root, "fhd")
    # 먼저 fhd 폴더에 남아있는 웹피 파일 중 원본에 없는 파일을 삭제
    clean_fhd_folder(fhd_folder, source_files)

    # fhd 폴더가 없으면 생성
    if not os.path.exists(fhd_folder):
        os.makedirs(fhd_folder)

    # tqdm으로 파일 처리 과정을 진행 상황 표시
    for file in tqdm(files, desc=f"Processing: {root}", leave=False):
        # 소문자 확장자 비교
        ext = os.path.splitext(file)[1].lower()
        if ext in (".jpeg", ".jpg", ".png"):
            file_path = os.path.join(root, file)
            output_name = os.path.splitext(file)[0] + ".webp"
            output_path = os.path.join(fhd_folder, output_name)
            '''
            # 이미 변환된 파일이 있으면 스킵
            if os.path.exists(output_path):
                # 필요 시 진행 상황 출력
                # print(f"Skipping: {output_path} (already exists)")
                continue
            '''
            # 이미지 변환 진행
            resize_image(file_path, output_path, max_size=1920)

print("이미지 리사이즈 및 WebP 변환 작업이 완료되었습니다!")


이미지 리사이즈 및 WebP 변환 작업이 완료되었습니다!
